# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [103]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

In [104]:
nhl_df=pd.read_csv("nhl.csv")
cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


In [105]:
def removenote(cities, tag):
    templist = []
    for a in cities[tag]:
        templist.append(a.split('[')[0])
    cities[tag] = templist
    return cities

In [106]:
cities = removenote(cities,'NHL')
# cities = removenote(cities,'NFL')
# cities = removenote(cities,'MLB')
# cities = removenote(cities,'NBA')

cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [107]:
cities = cities.rename(columns={'Population (2016 est.)[8]':'Population 2016'})
cities.head()

,Metropolitan area,Population 2016,NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [108]:
cities = cities.drop(['NFL','MLB','NBA'],axis=1)

In [109]:
cities[cities['Metropolitan area'] == 'New York City']['NHL'][0]

'RangersIslandersDevils'

In [110]:
s = 'RangersIslandersDevils'

s = 'Red Wings'
pattern = '([A-Z][^\s][a-z]*)'
x = re.findall(pattern,cities[cities['Metropolitan area'] == 'New York City']['NHL'][0])
x

['Rangers', 'Islanders', 'Devils']

In [111]:
# replace new york city teams
pattern = '([A-Z][^\s][a-z]*)'
x = re.findall(pattern,cities[cities['Metropolitan area'] == 'New York City']['NHL'][0])
cities.iloc[0]['NHL'] = x
cities.head()

,Metropolitan area,Population 2016,NHL
0,New York City,20153634,"[Rangers, Islanders, Devils]"
1,Los Angeles,13310447,KingsDucks
2,San Francisco Bay Area,6657982,Sharks
3,Chicago,9512999,Blackhawks
4,Dallas–Fort Worth,7233323,Stars


In [112]:
# replace LA teams
pattern = '([A-Z][^\s][a-z]*)'
x = re.findall(pattern,cities[cities['Metropolitan area'] == 'Los Angeles']['NHL'][1])
cities.iloc[1]['NHL'] = x
cities.head()

,Metropolitan area,Population 2016,NHL
0,New York City,20153634,"[Rangers, Islanders, Devils]"
1,Los Angeles,13310447,"[Kings, Ducks]"
2,San Francisco Bay Area,6657982,Sharks
3,Chicago,9512999,Blackhawks
4,Dallas–Fort Worth,7233323,Stars


In [113]:
cities = cities.explode('NHL')

In [114]:
cities.head()

,Metropolitan area,Population 2016,NHL
0,New York City,20153634,Rangers
0,New York City,20153634,Islanders
0,New York City,20153634,Devils
1,Los Angeles,13310447,Kings
1,Los Angeles,13310447,Ducks


In [126]:
# cities['NHL'].to_list()

In [115]:
# remove the unnecessary first row
nhl_df = nhl_df.drop(0)
nhl_df.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL
5,Detroit Red Wings,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL


In [118]:
nhl_df['team'] = nhl_df['team'].replace('*','')

In [119]:
nhl_df['team']

1      Tampa Bay Lightning*
2            Boston Bruins*
3      Toronto Maple Leafs*
4          Florida Panthers
5         Detroit Red Wings
               ...         
166      Los Angeles Kings*
167         Phoenix Coyotes
168       Vancouver Canucks
169          Calgary Flames
170         Edmonton Oilers
Name: team, Length: 170, dtype: object

In [122]:
nhl_df['team'] = nhl_df['team'].apply(lambda x: x.replace('*',''))
nhl_df

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
1,Tampa Bay Lightning,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL
5,Detroit Red Wings,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,Los Angeles Kings,82,46,28,8,100,.610,206,174,0.40,0.01,.549,38,2014,NHL
167,Phoenix Coyotes,82,37,30,15,89,.543,216,231,-0.16,0.03,.457,31,2014,NHL
168,Vancouver Canucks,82,36,35,11,83,.506,196,223,-0.30,0.03,.451,31,2014,NHL
169,Calgary Flames,82,35,40,7,77,.470,209,241,-0.35,0.04,.402,28,2014,NHL


In [235]:
# nhl_df[nhl_df['year'] == 2018]

In [127]:
all_nhl_teams = set(nhl_df['team'].to_list())
all_nhl_teams

{'Anaheim Ducks',
 'Arizona Coyotes',
 'Atlantic Division',
 'Boston Bruins',
 'Buffalo Sabres',
 'Calgary Flames',
 'Carolina Hurricanes',
 'Central Division',
 'Chicago Blackhawks',
 'Colorado Avalanche',
 'Columbus Blue Jackets',
 'Dallas Stars',
 'Detroit Red Wings',
 'Edmonton Oilers',
 'Florida Panthers',
 'Los Angeles Kings',
 'Metropolitan Division',
 'Minnesota Wild',
 'Montreal Canadiens',
 'Nashville Predators',
 'New Jersey Devils',
 'New York Islanders',
 'New York Rangers',
 'Ottawa Senators',
 'Pacific Division',
 'Philadelphia Flyers',
 'Phoenix Coyotes',
 'Pittsburgh Penguins',
 'San Jose Sharks',
 'St. Louis Blues',
 'Tampa Bay Lightning',
 'Toronto Maple Leafs',
 'Vancouver Canucks',
 'Vegas Golden Knights',
 'Washington Capitals',
 'Winnipeg Jets'}

In [137]:
# Now fix the team names in cities df
def findorgname(team):
    if team == '':
        return '-'
    for x in all_nhl_teams:
        if team in x:
            return x
    return '-'
temp_nhl_list = []
for team in cities['NHL']:
    temp_nhl_list.append(findorgname(team))

# change the nhl columns in cities df
cities['NHL'] = temp_nhl_list
cities['NHL']

0          New York Rangers
0        New York Islanders
0         New Jersey Devils
1         Los Angeles Kings
1             Anaheim Ducks
2           San Jose Sharks
3        Chicago Blackhawks
4              Dallas Stars
5       Washington Capitals
6       Philadelphia Flyers
7             Boston Bruins
8            Minnesota Wild
9        Colorado Avalanche
10         Florida Panthers
11          Phoenix Coyotes
12        Detroit Red Wings
13      Toronto Maple Leafs
14                        -
15                        -
16      Tampa Bay Lightning
17      Pittsburgh Penguins
18                        -
19                        -
20                        -
21                        -
22          St. Louis Blues
23                        -
24                        -
25                        -
26      Nashville Predators
27                        -
28                        -
29           Buffalo Sabres
30       Montreal Canadiens
31        Vancouver Canucks
32                  

In [141]:
nhl_df = nhl_df.set_index('team')
nhl_df.head()

,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
team,,,,,,,,,,,,,,
Tampa Bay Lightning,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
Boston Bruins,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
Toronto Maple Leafs,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL
Detroit Red Wings,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL


In [148]:
cities = cities.rename({'NHL':'team'}, axis = 1)
cities = cities.set_index('team')
cities.head()

,Metropolitan area,Population 2016
team,,
New York Rangers,New York City,20153634
New York Islanders,New York City,20153634
New Jersey Devils,New York City,20153634
Los Angeles Kings,Los Angeles,13310447
Anaheim Ducks,Los Angeles,13310447


In [151]:
merged = pd.merge(nhl_df, cities, how='inner',left_index = True, right_index=True)
merged.head()

,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League,Metropolitan area,Population 2016
team,,,,,,,,,,,,,,,,
Anaheim Ducks,82,44,25,13,101,.616,235,216,0.24,0.01,.555,40,2018,NHL,Los Angeles,13310447
Anaheim Ducks,82,46,23,13,105,.640,223,200,0.24,-0.04,.561,43,2017,NHL,Los Angeles,13310447
Anaheim Ducks,82,46,25,11,103,.628,218,192,0.27,-0.05,.567,43,2017,NHL,Los Angeles,13310447
Anaheim Ducks,82,51,24,7,109,.665,236,226,0.10,-0.03,.604,43,2015,NHL,Los Angeles,13310447
Anaheim Ducks,82,54,20,8,116,.707,266,209,0.68,-0.01,.677,51,2014,NHL,Los Angeles,13310447


In [190]:
# removing all other than year 2018
merged = merged[merged['year'] == 2018]
merged.head()

,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League,Metropolitan area,Population 2016
team,,,,,,,,,,,,,,,,
Anaheim Ducks,82,44,25,13,101,.616,235,216,0.24,0.01,.555,40,2018,NHL,Los Angeles,13310447
Boston Bruins,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL,Boston,4794447
Buffalo Sabres,82,25,45,12,62,.378,199,280,-0.98,0.01,.311,24,2018,NHL,Buffalo,1132804
Calgary Flames,82,37,35,10,84,.512,218,248,-0.33,0.03,.470,35,2018,NHL,Calgary,1392609
Carolina Hurricanes,82,36,35,11,83,.506,228,256,-0.35,-0.01,.439,33,2018,NHL,Raleigh,1302946


In [203]:
area_indexed = merged.reset_index()
area_indexed.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League,Metropolitan area,Population 2016
0,Anaheim Ducks,82,44,25,13,101,.616,235,216,0.24,0.01,.555,40,2018,NHL,Los Angeles,13310447
1,Boston Bruins,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL,Boston,4794447
2,Buffalo Sabres,82,25,45,12,62,.378,199,280,-0.98,0.01,.311,24,2018,NHL,Buffalo,1132804
3,Calgary Flames,82,37,35,10,84,.512,218,248,-0.33,0.03,.470,35,2018,NHL,Calgary,1392609
4,Carolina Hurricanes,82,36,35,11,83,.506,228,256,-0.35,-0.01,.439,33,2018,NHL,Raleigh,1302946


In [204]:
area_indexed = area_indexed.drop(area_indexed.columns.difference(['W','L','team','Population 2016','Metropolitan area']),axis=1)

In [205]:
area_indexed = area_indexed.astype({'W':int, 'L':int,'Population 2016':int})

In [206]:
area_indexed[0:]

,team,W,L,Metropolitan area,Population 2016
0,Anaheim Ducks,44,25,Los Angeles,13310447
1,Boston Bruins,50,20,Boston,4794447
2,Buffalo Sabres,25,45,Buffalo,1132804
3,Calgary Flames,37,35,Calgary,1392609
4,Carolina Hurricanes,36,35,Raleigh,1302946
5,Chicago Blackhawks,33,39,Chicago,9512999
6,Colorado Avalanche,43,30,Denver,2853077
7,Columbus Blue Jackets,45,30,Columbus,2041520
8,Dallas Stars,42,32,Dallas–Fort Worth,7233323
9,Detroit Red Wings,30,39,Detroit,4297617


In [217]:
area_indexed.shape

(30, 5)

In [202]:
# I was trying to group them without removing other years
# resulting = []
# for group, frame in area_indexed.groupby('Metropolitan area'):
#     print(group)
#     population = frame['Population 2016'][0]
#     win = np.sum(frame['W'])
#     print('\tWon matches are',win)    
#     loss = np.sum(frame['L'])
#     print('\tLost matches are',loss)
#     print('\twin/loss ratio is ',win/(loss+win))
#     print('\tPopulation is ',population)

In [270]:
# group by city, sum up the win loss
a = 0
win_loss_by_region = []
population_by_region = []
for group, frame in area_indexed.groupby('Metropolitan area'):
#     print('\n',group,'----------')
#     print('\n',frame)
    win = np.sum(frame['W'])
    loss = np.sum(frame['L'])
    win_loss_by_region.append(win/(loss+win))

    population = frame['Population 2016'].iloc[0]
    population_by_region.append(population)
#     print('len {} and type {}'.format(len(population),type(population)))
#     if len(population) >1:
#         population = frame['Population 2016'][0]
#     print(group, win, loss, population)
    a+=1
# print(win_loss_by_region)
# print(population_by_region)
print('len',len(population_by_region))
print(stats.pearsonr(population_by_region, win_loss_by_region))

len 27
(0.01337397844453335, 0.9472124883027294)


In [291]:
cities[:]

,Metropolitan area,Population 2016
team,,
New York Rangers,New York City,20153634
New York Islanders,New York City,20153634
New Jersey Devils,New York City,20153634
Los Angeles Kings,Los Angeles,13310447
Anaheim Ducks,Los Angeles,13310447
San Jose Sharks,San Francisco Bay Area,6657982
Chicago Blackhawks,Chicago,9512999
Dallas Stars,Dallas–Fort Worth,7233323
Washington Capitals,"Washington, D.C.",6131977


In [304]:

def removenote(cities, tag):
    templist = []
    for a in cities[tag]:
        templist.append(a.split('[')[0])
    cities[tag] = templist
    return cities

# Now fix the team names in cities df
def findorgname(team,all_nhl_teams):
    if team == '':
        return '-'
    for x in all_nhl_teams:
        if team in x:
            return x
    return '-'

nhl_df=pd.read_csv("nhl.csv")
cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    global cities, nhl_df
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    cities = removenote(cities,'NHL')
    cities = cities.rename(columns={'Population (2016 est.)[8]':'Population 2016'})
    cities = cities.drop(['NFL','MLB','NBA'],axis=1)
#     cities[cities['Metropolitan area'] == 'New York City']['NHL'][0]


    
    # replace new york city teams
    pattern = '([A-Z][^\s][a-z]*)'
    x = re.findall(pattern,cities[cities['Metropolitan area'] == 'New York City']['NHL'][0])
    cities.iloc[0]['NHL'] = x
    
    # replace LA teams
    pattern = '([A-Z][^\s][a-z]*)'
    x = re.findall(pattern,cities[cities['Metropolitan area'] == 'Los Angeles']['NHL'][1])
    cities.iloc[1]['NHL'] = x
    
    
#     nhl_df = nhl_df.drop(0)
    
    nhl_df['team'] = nhl_df['team'].apply(lambda x: x.replace('*',''))
    
    all_nhl_teams = set(nhl_df['team'].to_list())
    cities = cities.explode('NHL')
    
    temp_nhl_list = []
    for team in cities['NHL']:
        actual_name = findorgname(team,all_nhl_teams)
        if actual_name == 'Phoenix Coyotes':
            actual_name = 'Arizona Coyotes'
        temp_nhl_list.append(actual_name)

    # change the nhl columns in cities df
    cities['NHL'] = temp_nhl_list
    nhl_df = nhl_df.set_index('team')
    
    cities = cities.rename({'NHL':'team'}, axis = 1)
    cities = cities.set_index('team')
    
    merged = pd.merge(cities,nhl_df, how='inner',left_index = True, right_index=True)
    merged = merged[merged['year'] == 2018]
    area_indexed = merged.reset_index()
    area_indexed = area_indexed.drop(area_indexed.columns.difference(['W','L','team','Population 2016','Metropolitan area']),axis=1)
    area_indexed = area_indexed.astype({'W':int, 'L':int,'Population 2016':int})
    area_indexed['win_loss_ratio'] = area_indexed['W']/(area_indexed['W']+area_indexed['L'])
    
    # group by city, sum up the win loss
    a = 0
    for group, frame in area_indexed.groupby('Metropolitan area'):
#         win = np.sum(frame['W'])
#         loss = np.sum(frame['L'])
        win_loss_by_region.append(np.mean(frame['win_loss_ratio']))

        population = frame['Population 2016'].iloc[0]
        population_by_region.append(population)
        a+=1

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)

In [305]:
nhl_correlation()

(0.012486162921209923, 0.9497182859911781)

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NFL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values